# __State Efficient Ansatz__

## Method explication


In many cases when we try to find the optimal solution of an optimization problem the optimizator could lead to a shallow space where the gradient descent vanishes, known as Barren Plateau (BP). Causes of BP are plenty studied in the literature, the most knwon sources are:   __ansatz__, __initial parameters__, __cost function__, __deep of circuit__, __hardware noise__, between others. In this notebook we will tackle the problem induced by the choice of a "bad" ansatz.

In general the chosen ansatz is a universal Parametrized Quantum Circuit (PQC) since it has a great expresibility, but in contrast has a poor trainability since the search space is the complete Hilbert space of the ansatz system. Xiu Liu et al. propose a new kind of ansatz call __State Efficient Ansatz__ (SEA) that removes some redundancy between the universal unitary and universal pure quantum states [1].

The next esqueme compares the case of a universal ansatz state and the state efficient ansatz.

<center><img src="Conceptualization.jpg" alt="Conceptualization" style="width:500px;"/><center>

 SEA can represent pure and low bipartite entangled quantum states with fewer parameters [1]. That make SEA an efficient way to solve state-oriented tasks 

## __Example:__ Ground State of BeH2

We solve the ground state of BeH2. For that we reduce the numbers of qubits from 14 to 6 omitting the electrons in the bulk and orbitals that correspond to very exited states.

<center><img src="beh2.jpg" alt="beh2" style="width:300px;"/><center>

As usual we import some standar libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import transpile, QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.utils import QuantumInstance

KeyboardInterrupt: 

And import a circuit maker that create an efficient SU(2) gate

In [ ]:
from qiskit.circuit.library import EfficientSU2

Then we import the Quantum Barren Plateaus (qubap) module in the qiskit version

In [ ]:
import sys
sys.path.append('../..')

from qubap.qiskit.luciano.variational_algorithms import VQE, classical_solver, energy_evaluation
from qubap.qiskit.luciano.costfunc_barren_plateaus import paulistrings2hamiltonian 
from qubap.qiskit.felipe.state_efficient_ansatz import ansatz_constructor #SAE

Now we set our backend with the number of shots

In [ ]:
num_shots = 2**10
num_iters = 250
quantum_instance = QuantumInstance( AerSimulator() , shots = num_shots )

After that we define the number of qubits and cast the BeH2 information from a numpy array.

In [ ]:
molecule = np.load( 'molecule_BeH2_6.npy', allow_pickle=True )[0]

The molecule file contains the next information

In [ ]:
molecule.keys()

We are interested in the 'hamiltonian' and the 'inital_state' data. The 'initial_state' contain the Hartree Fock information. Then we transform the numpy 'hamiltonian' of the BeH2 with *paulistrings2hamiltonian* function to a quantum circuit.

In [ ]:
pauli_strings, coeffs = molecule['hamiltonian']
hamiltonian = paulistrings2hamiltonian( pauli_strings, coeffs )
num_qubits = hamiltonian.num_qubits

In [ ]:
hf= QuantumCircuit.from_qasm_str(molecule['initial_state'])

We first find the optimal value with a classical solver

In [ ]:
exact_energy = classical_solver( hamiltonian ).eigenvalue
exact_energy

Then we set the deep (rep) of the general efficient SU(2) circuit with circular entanglement

In [ ]:
rep       = 2
ansatz_SU = EfficientSU2( num_qubits, ['ry','rz'], 'circular', rep ).decompose()
ansatz_SU.compose(hf, inplace=True)
t_ansatz_SU = transpile(ansatz_SU)
#ansatz_SU.draw('mpl') 

Finally we define the number of iterations of the SPSA optimizer and some random parameters to inizializate the VQE

In [ ]:
pars_SU     = np.zeros( t_ansatz_SU.num_parameters ) + 0.01
#pars_SU     = np.random.rand(t_ansatz_SU.num_parameters) * np.pi
results_SU  = VQE( hamiltonian,  t_ansatz_SU,  pars_SU, num_iters, quantum_instance, iter_start=1000 )

In [ ]:
data_SU = [ energy_evaluation( hamiltonian, t_ansatz_SU, x, AerSimulator(method='statevector') ) for x in results_SU['x'] ]

Now we plot the evaluation of the hamiltonian a long the iterations and the optimal value of the problem

In [ ]:
plt.plot(results_SU['fx'], label='general SU')
plt.plot(data_SU, label='data SU')
plt.hlines(exact_energy.real, 0,200, 'r', label='optimal solution')
plt.legend()

Now we construst our SEA circuit with the Hartree Fock state

In [ ]:
ansatz_SEA   = ansatz_constructor( num_qubits, deep=[rep,rep,rep] )
ansatz_SEA.compose(hf, inplace=True)
t_ansatz_SEA = transpile(ansatz_SEA)
ansatz_SEA.decompose().draw( 'mpl' )

Next we transpile it to make the circuit hardware efficient and perform the VQE with SEA

In [ ]:
npars_SEA = t_ansatz_SEA.num_parameters
pars_SEA  = np.zeros( t_ansatz_SEA.num_parameters ) + 0.01
#pars_SEA      = np.random.rand(t_ansatz_SEA.num_parameters) * np.pi
results_SEA = VQE( hamiltonian, t_ansatz_SEA, pars_SEA, num_iters, quantum_instance, iter_start=1000 )

In [ ]:
data_SEA = [ energy_evaluation( hamiltonian, t_ansatz_SEA, x, AerSimulator(method='statevector') ) for x in results_SEA['x'] ]

Now we plot both the efficient SU(2) circuit and the State Efficient Ansatz, with the optimal value of the problem

In [ ]:
plt.plot( results_SU['fx'], label='SU' )
plt.plot( results_SEA['fx'], label='SEA' )
plt.hlines( exact_energy.real, 0, num_iters, 'r', label='optimal solution' )
plt.legend(  )

In [ ]:
plt.plot( data_SU, label='SU' )
plt.plot( data_SEA, label='SEA' )
plt.hlines( exact_energy.real, 0, num_iters, 'r', label='optimal solution' )
plt.legend(  )

We can see that using the SEA aproach the barren plateau is over pass.